In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
from dowhy import CausalModel
import networkx as nx
from dowhy.utils import networkx_plotting

In [2]:
df = pd.read_excel('Sales Analysis.xlsx')

In [3]:
df.head()

,Order ID,Customer Name,Product,Region,"Sales ('000, USD)",Registered At,Purchased At,Refunded,Origin
0,1,Jones,A,South,500,2018-01-12 02:00:00,2018-06-12 18:59:00,False,referral
1,2,Kivell,B,East,400,2018-01-12 12:23:00,2018-06-12 19:54:00,False,display
2,3,James,G,North,100,2018-01-12 13:49:00,2018-06-13 16:37:00,False,social
3,4,Jack,F,West,200,2018-01-12 16:07:00,2018-06-14 14:30:00,False,social
4,5,Anny,B,Central,250,2018-01-12 18:25:00,2018-06-14 15:43:00,True,other


In [55]:
df['Purchased At'].nunique()

In [4]:
df.dtypes

Order ID                      int64
Customer Name                object
Product                      object
Region                       object
Sales ('000, USD)             int64
Registered At        datetime64[ns]
Purchased At         datetime64[ns]
Refunded                       bool
Origin                       object
dtype: object

In [5]:
df.rename(columns = {"Sales ('000, USD)":"Sales"},inplace = True)

In [6]:
df['Customer Name'] = df['Customer Name'].astype(str)

df['Origin'] = df['Origin'].astype(str)

df['Product'] = df['Product'].astype(str)

In [7]:
df.dtypes

Order ID                  int64
Customer Name            object
Product                  object
Region                   object
Sales                     int64
Registered At    datetime64[ns]
Purchased At     datetime64[ns]
Refunded                   bool
Origin                   object
dtype: object

In [8]:
df.describe()

,Order ID,Sales
count,150.000000,150.000000
mean,76.040000,289.033333
std,44.358346,127.018860
min,1.000000,100.000000
25%,38.250000,200.000000
50%,75.500000,300.000000
75%,113.750000,350.000000
max,190.000000,550.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       150 non-null    int64         
 1   Customer Name  150 non-null    object        
 2   Product        150 non-null    object        
 3   Region         150 non-null    object        
 4   Sales          150 non-null    int64         
 5   Registered At  150 non-null    datetime64[ns]
 6   Purchased At   150 non-null    datetime64[ns]
 7   Refunded       150 non-null    bool          
 8   Origin         150 non-null    object        
dtypes: bool(1), datetime64[ns](2), int64(2), object(4)
memory usage: 9.7+ KB


In [10]:
df['Origin'].value_counts()

social            35
organic_search    31
paid_search       28
unknown           16
email             14
direct_traffic    13
referral           6
display            4
other              3
Name: Origin, dtype: int64

In [11]:
df.groupby('Region')['Sales'].sum()

Region
Central     7755
East       11960
North       6775
South       7695
West        9170
Name: Sales, dtype: int64

In [12]:
df.groupby('Origin')['Refunded'].sum()

Origin
direct_traffic    0
display           0
email             0
organic_search    1
other             1
paid_search       2
referral          0
social            1
unknown           2
Name: Refunded, dtype: int64

In [67]:
origins = Origin_info['Origin']
total_sales = Origin_info['Total Sales']

labels = origins + ['Total Sales']

sources = list(range(len(origins)))  
targets = [len(origins)] * len(origins)  

fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=total_sales
    )
))

fig.update_layout(title_text="Sales Funnel", font_size=10)
fig.show()

In [13]:
Origin_info = df.groupby('Origin').agg({'Region':'count','Sales':'sum','Refunded':'sum'}).reset_index().rename(columns = {'Region':'Count','Sales':'Total Sales','Refunded':'Products Refunded'}).sort_values(by = 'Total Sales',ascending = False)
Origin_info['Average Order Value'] = round(Origin_info['Total Sales']/Origin_info['Count'],2)
Origin_info['Percentage of Products Returned'] = round(Origin_info['Products Refunded']/Origin_info['Count']*100,2)

# df_melted = Origin_info.melt(id_vars='Origin', value_vars=['Count', 'Total Sales'], var_name='Metric', value_name='Value')

In [78]:
Origin_info.sort_values(by = 'Average Order Value', ascending = False)

,Origin,Count,Total Sales,Products Refunded,Average Order Value,Percentage of Products Returned
8,unknown,16,5125,2,320.31,12.50
5,paid_search,28,8475,2,302.68,7.14
3,organic_search,31,9350,1,301.61,3.23
7,social,35,10280,1,293.71,2.86
2,email,14,3990,0,285.00,0.00
1,display,4,1040,0,260.00,0.00
0,direct_traffic,13,3160,0,243.08,0.00
6,referral,6,1335,0,222.50,0.00
4,other,3,600,1,200.00,33.33


In [16]:
df['Product'].value_counts()

B    28
G    24
F    24
C    22
D    20
A    18
E    14
Name: Product, dtype: int64

In [19]:
df[df['Refunded']].groupby('Region').agg({'Region':'count','Product':'unique'})

,Region,Product
Region,,
Central,3,"[B, F, A]"
East,2,"[F, B]"
North,1,[C]
South,1,[B]


In [22]:
df[df['Refunded']].groupby('Product')['Region'].unique()

Product
A                 [Central]
B    [Central, East, South]
C                   [North]
F           [Central, East]
Name: Region, dtype: object

In [23]:
df['Registered At'].dt.month.value_counts()

2    82
1    68
Name: Registered At, dtype: int64

In [25]:
# df[df['Refunded']].groupby('Region')['Product'].count()

In [26]:
value_counts_df = df.groupby('Region')['Origin'].value_counts().reset_index(name='count')
idx = value_counts_df.groupby('Region')['count'].idxmax()
value_counts_df.loc[idx]

,Region,Origin,count
0,Central,social,9
8,East,organic_search,11
16,North,organic_search,7
24,South,social,6
33,West,paid_search,7


In [27]:
df['Origin'].unique()

array(['referral', 'display', 'social', 'other', 'paid_search', 'unknown',
       'email', 'direct_traffic', 'organic_search'], dtype=object)

In [28]:
# Define the mapping
mapping = {
    'direct_traffic': 'Organic',
    'referral': 'Organic',
    'organic_search': 'Organic'
}

# Apply the mapping to the 'Origin' column
df['Origin'] = df['Origin'].apply(lambda x: mapping.get(x, 'Paid'))

In [29]:
df['Origin'].value_counts()

Paid       100
Organic     50
Name: Origin, dtype: int64

In [75]:
df.groupby(['Region','Origin'])['Sales'].sum().reset_index().sort_values(by = ['Region','Sales'],ascending = [True,False])

# In all regions Sales for the paid channel was more  

,Region,Origin,Sales
1,Central,Paid,5185
0,Central,Organic,2570
3,East,Paid,8060
2,East,Organic,3900
5,North,Paid,3880
4,North,Organic,2895
7,South,Paid,4685
6,South,Organic,3010
9,West,Paid,7700
8,West,Organic,1470


In [53]:
df['Time to Purchase (days)'] = (df['Purchased At'] - df['Registered At']).dt.days

In [52]:
df.groupby('Customer Name').agg(
    Min_Time_to_Purchase = ('Time to Purchase (days)', 'min'),
    Max_Time_to_Purchase = ('Time to Purchase (days)', 'max'),
    Product_Bought = ('Time to Purchase (days)', 'count')
).reset_index()

,Customer Name,Min_Time_to_Purchase,Max_Time_to_Purchase,Product_Bought
0,Amy,150,262,7
1,Andrew,152,221,8
2,Anny,152,224,7
3,Daisy,152,223,6
4,Elena,186,223,2
5,Gill,150,263,7
6,Isle,218,218,1
7,Jack,152,249,14
8,James,152,226,9
9,Jardine,152,251,13


In [48]:
df['rnk'] = df.groupby('Customer Name')['Purchased At'].rank()

In [49]:
# pivot_df = df.pivot_table(index='Customer Name', columns='rnk', values='Purchased At')

# pivot_df.columns = ['Rank 1', 'Rank 2']

# pivot_df['Difference'] = pivot_df['Rank 2'] - pivot_df['Rank 1']

# pivot_df = pivot_df.reset_index()

# print(pivot_df)

rank1 = df[df['rnk'] == 1].rename(columns={'Purchased At': 'Purchased At Rank 1'})
rank2 = df[df['rnk'] == 2].rename(columns={'Purchased At': 'Purchased At Rank 2'})

merged_df = pd.merge(rank1[['Customer Name', 'Purchased At Rank 1']],
                     rank2[['Customer Name', 'Purchased At Rank 2']],
                     on='Customer Name',
                     how='outer')

merged_df['Difference'] = merged_df['Purchased At Rank 2'] - merged_df['Purchased At Rank 1']

merged_df['Difference'] = merged_df['Difference'].astype(str)

merged_df['Difference'] = np.where(merged_df['Purchased At Rank 2'].isna(), 
                               'Only one item purchased', 
                               merged_df['Difference'])

merged_df

,Customer Name,Purchased At Rank 1,Purchased At Rank 2,Difference
0,Jones,2018-06-12 18:59:00,2018-06-18 13:51:00,5 days 18:52:00
1,Kivell,2018-06-12 19:54:00,2018-06-19 12:36:00,6 days 16:42:00
2,James,2018-06-13 16:37:00,2018-06-18 13:58:00,4 days 21:21:00
3,Jack,2018-06-14 14:30:00,2018-06-29 20:16:00,15 days 05:46:00
4,Anny,2018-06-14 15:43:00,2018-06-26 17:13:00,12 days 01:30:00
5,Gill,2018-06-14 18:51:00,2018-06-26 12:51:00,11 days 18:00:00
6,Amy,2018-06-15 14:04:00,2018-06-25 19:33:00,10 days 05:29:00
7,Morgan,2018-06-15 19:49:00,2018-06-25 18:42:00,9 days 22:53:00
8,Smith,2018-06-15 21:32:00,2018-07-03 20:17:00,17 days 22:45:00
9,Sorvina,2018-06-18 12:46:00,2018-06-19 14:08:00,1 days 01:22:00


In [45]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('Sales ~ C(Origin)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

                 sum_sq     df         F    PR(>F)
C(Origin)  1.104133e+04    1.0  0.682904  0.409919
Residual   2.392894e+06  148.0       NaN       NaN


In [46]:
df.corr(numeric_only = True)

,Order ID,Sales,Refunded,Time to Purchase (days)
Order ID,1.000000,-0.000160,-0.055964,0.968596
Sales,-0.000160,1.000000,-0.010794,0.009753
Refunded,-0.055964,-0.010794,1.000000,-0.041408
Time to Purchase (days),0.968596,0.009753,-0.041408,1.000000


In [47]:
correlation_matrix = df.corr(numeric_only=True)

fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    zmin=-1,
    zmax=1,
    colorbar=dict(title='Correlation'),
))

fig.update_layout(
    title='Correlation Matrix',
    xaxis_title='Variables',
    yaxis_title='Variables',
    xaxis=dict(tickangle=-45),
    yaxis=dict(tickangle=0),
)

fig.show()

In [ ]:
# # Correlation Matrix
# correlation_matrix = df.corr(numeric_only=True)

# # Plot Correlation Matrix using Seaborn
# plt.figure(figsize=(10, 8))  # Set the figure size
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f")
# plt.title('Correlation Matrix', fontsize=16)
# plt.xticks(rotation=45, ha='right')  # Rotate the x-axis labels for better readability
# plt.yticks(rotation=0)  # Keep y-axis labels horizontal
# plt.show()

In [43]:
df['Customer Name'].value_counts()

Kivell      15
Smith       15
Jack        14
Jardine     13
Morgan      11
Thompson    10
Sorvina      9
James        9
Andrew       8
Jones        7
Amy          7
Gill         7
Anny         7
Daisy        6
Rebecca      4
Mack         3
Elena        2
Jonshon      1
Isle         1
Mathew       1
Name: Customer Name, dtype: int64

In [44]:
df.groupby('Customer Name')['Region'].nunique()

# All Customers Bought products from one region

Customer Name
Amy         1
Andrew      1
Anny        1
Daisy       1
Elena       1
Gill        1
Isle        1
Jack        1
James       1
Jardine     1
Jones       1
Jonshon     1
Kivell      1
Mack        1
Mathew      1
Morgan      1
Rebecca     1
Smith       1
Sorvina     1
Thompson    1
Name: Region, dtype: int64

In [ ]:
# df.groupby(['Customer Name','Region'])['Order ID'].count().sort_values(ascending = False)

In [38]:
df[df['Refunded']].groupby(['Refunded', 'Product']).size().reset_index(name='count')

,Refunded,Product,count
0,True,A,1
1,True,B,3
2,True,C,1
3,True,F,2


In [39]:
df[df['Refunded']].groupby(['Region','Product']).size().reset_index(name='count')

,Region,Product,count
0,Central,A,1
1,Central,B,1
2,Central,F,1
3,East,B,1
4,East,F,1
5,North,C,1
6,South,B,1


In [40]:
df[df['Refunded']].groupby('Region').size().reset_index(name='count')

,Region,count
0,Central,3
1,East,2
2,North,1
3,South,1


In [41]:
df_agg = df.groupby('Product').agg({'Sales':'sum','Product':'count'}).rename(columns = {'Sales':'Total Sales','Product':'Number of Orders'})

In [42]:
avg_order_value = df_agg['Total Sales']/df_agg['Number of Orders']

avg_order_value

Product
A    411.388889
B    349.285714
C    274.772727
D    248.000000
E    318.571429
F    263.125000
G    182.916667
dtype: float64

In [35]:
# Checking if a given product has the same value in the same region and how it differs across different regions

regions = df['Region'].unique()
products = df['Product'].unique()

for region in regions:
    for product in products:
        sales_values = df[(df['Region'] == region)&(df['Product']==product)]['Sales'].values
        print(f"Region : {region}")
        print(f"Product : {product}")
        print("Sales Values:", sales_values)
        print()

Region : South
Product : A
Sales Values: [500 500 500 235]

Region : South
Product : B
Sales Values: [400 350 400 400]

Region : South
Product : G
Sales Values: [350 100 100]

Region : South
Product : F
Sales Values: [500 200 200 200 200 410]

Region : South
Product : D
Sales Values: [350]

Region : South
Product : E
Sales Values: [220]

Region : South
Product : C
Sales Values: [110 300 520 350 300]

Region : East
Product : A
Sales Values: [350 120 500 500 500]

Region : East
Product : B
Sales Values: [400 320 210 400 350 300 400 210 400 135]

Region : East
Product : G
Sales Values: [550 110 210 350 120 130 105]

Region : East
Product : F
Sales Values: [350 200 200 320 200 335]

Region : East
Product : D
Sales Values: [250 250]

Region : East
Product : E
Sales Values: [200 220 550 550 205]

Region : East
Product : C
Sales Values: [320 300 300 320 220]

Region : North
Product : A
Sales Values: [520 210 300]

Region : North
Product : B
Sales Values: [400 400 320 350]

Region : North
Prod

In [36]:
for product in products :
    print(df[(df['Region']=='Central')&(df['Product']==product)][['Product','Sales']]) 
    
# Only Product G and F in the Central region has the same price for each product 

    Product  Sales
79        A    350
134       A    500
137       A    520
    Product  Sales
4         B    250
8         B    400
17        B    310
52        B    300
73        B    325
96        B    400
133       B    350
   Product  Sales
38       G    100
55       G    100
66       G    100
80       G    100
   Product  Sales
19       F    200
24       F    200
40       F    200
83       F    200
    Product  Sales
33        D    250
105       D    310
109       D    250
120       D    250
   Product  Sales
43       E    320
71       E    210
    Product  Sales
51        C    110
122       C    200
128       C    320
129       C    520
146       C    110


# Insights 

In [31]:
# 1. Total Sales
total_sales = df["Sales"].sum()

# 2. Sales by Region
sales_by_region = df.groupby('Region')["Sales"].sum().sort_values(ascending=False)

# 3. Sales by Product
sales_by_product = df.groupby('Product')["Sales"].sum().sort_values(ascending=False)

# 4. Number of Orders by Customer
orders_by_customer = df['Customer Name'].value_counts().head(10)

# 5. Average Sales per Order
average_sales_per_order = df["Sales"].mean()

{   
    "Total Sales (in '000 USD)": total_sales,
    "Sales by Region": sales_by_region,
    "Sales by Product": sales_by_product,
    "Top 10 Customers by Orders": orders_by_customer,
    "Average Sales per Order (in '000 USD)": average_sales_per_order,
}

{"Total Sales (in '000 USD)": 43355,
 'Sales by Region': Region
 East       11960
 West        9170
 Central     7755
 South       7695
 North       6775
 Name: Sales, dtype: int64,
 'Sales by Product': Product
 B    9780
 A    7405
 F    6315
 C    6045
 D    4960
 E    4460
 G    4390
 Name: Sales, dtype: int64,
 'Top 10 Customers by Orders': Kivell      15
 Smith       15
 Jack        14
 Jardine     13
 Morgan      11
 Thompson    10
 Sorvina      9
 James        9
 Andrew       8
 Jones        7
 Name: Customer Name, dtype: int64,
 "Average Sales per Order (in '000 USD)": 289.03333333333336}

In [64]:
# 6. Sales by Origin
sales_by_origin = df.groupby('Origin')["Sales"].sum().sort_values(ascending=False)

# 7. Percentage of Refunded Orders
refund_percentage = df['Refunded'].mean() * 100

# 8. Time taken from Registration to Purchase
# df['Time to Purchase (days)'] = (df['Purchased At'] - df['Registered At']).dt.days
average_time_to_purchase = df['Time to Purchase (days)'].mean()

# 9. Sales distribution over time (Registered At)
sales_over_time = df.groupby(df['Registered At'].dt.date)["Sales"].sum().reset_index()

# 10. Correlation between Refunds and Sales Amount
refund_sales_correlation = df['Refunded'].astype(int).corr(df["Sales"])

{    "Sales by Origin": sales_by_origin,
    "Refund Percentage": round(refund_percentage,2),
    "Average Time to Purchase (days)": round(average_time_to_purchase,2),
    "Sales Over Time": sales_over_time.head(), 
    "Correlation between Refunds and Sales": refund_sales_correlation
}

{'Sales by Origin': Origin
 Paid       29510
 Organic    13845
 Name: Sales, dtype: int64,
 'Refund Percentage': 4.67,
 'Average Time to Purchase (days)': 191.21,
 'Sales Over Time':   Registered At  Sales
 0    2018-01-12   1450
 1    2018-01-15   1550
 2    2018-01-16   1060
 3    2018-01-17    750
 4    2018-01-18   1180,
 'Correlation between Refunds and Sales': -0.010794088311416206}

1) Only Product G and F in the Central region has the same price for each product 
2) Origin does not have a statistically significant effect on the Sales based on the ANOVA test
3) Though sales that come through unidentified channels has the highest average order 
value it also has the highest percentage of products returned while 
in terms of absolute value socials have the highest sales with very low percentage of returns
4) All Customers Bought products from one region
5) In all regions Sales for the paid channel was more in comparison to organic channel
6) Johnson,Ile and Matthew are the only customers who have ordered once.

# Total Sales By Origin

In [65]:
px.bar(Origin_info, x = 'Origin', y = 'Total Sales',hover_data = ['Count','Average Order Value','Percentage of Products Returned'],text = 'Total Sales')

# Sales Trend

In [62]:
px.line(data_frame = sales_over_time, x = 'Registered At', y = 'Sales')

##### Testing Streamlit Code 

In [ ]:
st.title("Sales Data Analysis and Causal Inference")

st.header("Data Preview")
st.write(df.head())

st.header("Exploratory Data Analysis")

st.write(f"**Total Sales (in '000 USD):** {total_sales}")
st.write(f"**Average Sales per Order (in '000 USD):** {average_sales_per_order}")
st.write(f"**Refund Percentage:** {refund_percentage}%")
st.write(f"**Average Time to Purchase (days):** {average_time_to_purchase} days")

st.subheader("Sales by Region")
st.bar_chart(sales_by_region)

st.subheader("Sales by Product")
st.bar_chart(sales_by_product)

st.subheader("Top 10 Customers by Orders")
st.write(orders_by_customer)

st.subheader("Sales by Origin")
st.bar_chart(sales_by_origin)

st.subheader("Sales Over Time")
st.line_chart(sales_over_time)

st.header("Correlation Analysis")

correlation_matrix = df.corr(numeric_only=True)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f", ax=ax)
ax.set_title('Correlation Matrix')
st.pyplot(fig)

st.header("Causal Inference")

causal_model = CausalModel(
    data=df,
    treatment="Refunded",
    outcome="Sales",
    common_causes=["Time to Purchase (days)"]
)

st.subheader("Causal Model Graph")

fig, ax = plt.subplots()

causal_graph = causal_model._graph._graph  # Access the internal graph

pos = nx.spring_layout(causal_graph)  # or another layout for better spacing
nx.draw(causal_graph, pos, with_labels=True, node_color='lightblue', ax=ax, node_size=3000, font_size=10)

st.pyplot(fig)

identified_estimand = causal_model.identify_effect()
st.write("Identified Estimand:")
st.write(identified_estimand)

causal_estimate = causal_model.estimate_effect(identified_estimand, method_name="backdoor.linear_regression")
st.write("Causal Estimate:")
st.write(causal_estimate)

refutation = causal_model.refute_estimate(identified_estimand, causal_estimate, method_name="placebo_treatment_refuter")
st.write("Refutation Summary:")
st.write(refutation)

st.write("**Estimated Causal Effect:**", causal_estimate.value)
st.write("**Refutation (Placebo Treatment):**", refutation)